In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dense, Flatten, Input

In [2]:
pixels = 200
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])


data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      color_mode = 'grayscale',
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()   #hna a3tkd kan mafroud n7ot gwa el repeat el rkm el epoch kol el sites bt3ml kda

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Input size (200, 200)
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [3]:
NUM_OF_CLASSES = 29

In [4]:
mobile = tf.keras.applications.mobilenet.MobileNet(input_tensor=Input(shape=(200,200,3)))
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 100, 100, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 100, 100, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 100, 100, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 100, 100, 32)     128       
 ation)                                         

In [5]:
#i remove the first 7 layers and the last 6 layers and take the layers between them to form a model and merge later
x = tf.keras.models.Sequential(mobile.layers[7:-8]) #casting the layers to a sequential model 
x.build((None,100,100,32)) #build the layer to be a model and expecting input = (None,100,100,32)
x.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_pw_1 (Conv2D)          (None, 100, 100, 64)      2048      
                                                                 
 conv_pw_1_bn (BatchNormaliz  (None, 100, 100, 64)     256       
 ation)                                                          
                                                                 
 conv_pw_1_relu (ReLU)       (None, 100, 100, 64)      0         
                                                                 
 conv_pad_2 (ZeroPadding2D)  (None, 101, 101, 64)      0         
                                                                 
 conv_dw_2 (DepthwiseConv2D)  (None, 50, 50, 64)       576       
                                                                 
 conv_dw_2_bn (BatchNormaliz  (None, 50, 50, 64)       256       
 ation)                                               

<h1>We merged the input block and the middle block and the final layer

In [6]:
from tensorflow.keras.layers import Input, Dense, Flatten, ReLU, Conv2D, DepthwiseConv2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

# merge the layers (Input -> the middle model from above -> the final layer)

new_model = tf.keras.Sequential(
    [
        Input(shape=(200, 200, 1)),
        Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same'),
        BatchNormalization(),
        ReLU(),

        DepthwiseConv2D(kernel_size=3, strides=1, padding='same'),
        BatchNormalization(),
        ReLU(),

        x,

        GlobalAveragePooling2D(), #same as flatten but with much less parameters
        Dense(units= 29, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = 'softmax')
    ]
)


new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 100, 100, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 100, 100, 32)     320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 100, 100, 32)     128       
 hNormalization)                                                 
                                                      

In [12]:
for layer in new_model.layers[7:-2] :
    layer.trainable = False
new_model.build((None,)+IMAGE_SIZE+(3,))
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 100, 100, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 100, 100, 32)     320       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 100, 100, 32)     128       
 hNormalization)                                                 
                                                      

In [9]:
new_model.compile(optimizer= Adam(learning_rate= 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [10]:
steps_per_epoch = train_size  // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

new_model.fit(x = train_ds, validation_data=val_ds, epochs=2, verbose=1, steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps)

Epoch 1/2
4350/4350 [==============================] - 932s 211ms/step - loss: 0.1872 - accuracy: 0.9709 - val_loss: 0.0078 - val_accuracy: 1.0000
Epoch 2/2
4350/4350 [==============================] - 813s 187ms/step - loss: 0.0128 - accuracy: 0.9990 - val_loss: 0.0070 - val_accuracy: 1.0000


In [11]:
saved_model_path = "3-Saved_Model_With_TF1_GreyScale_WithoutTheLast9Layers"
tf.saved_model.save(new_model, saved_model_path)
new_model.save("MobileNetWithoutTheLast9Layers_GreyScale.h5")
print("Saved Model")

saved_model_path = "Saved_Model_With_TFWith_Keras_GreyScale_WithoutTheLast9Layers"
tf.keras.models.save_model(new_model, saved_model_path)

INFO:tensorflow:Assets written to: 3-Saved_Model_With_TF1_GreyScale_WithoutTheLast9Layers\assets


INFO:tensorflow:Assets written to: 3-Saved_Model_With_TF1_GreyScale_WithoutTheLast9Layers\assets


Saved Model


INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras_GreyScale_WithoutTheLast9Layers\assets


INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras_GreyScale_WithoutTheLast9Layers\assets


In [13]:
# Create the .tflite file

tflite_model_file = "3_withoutlast9Layers_Model.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model("3-Saved_Model_With_TF1_GreyScale_WithoutTheLast9Layers")

#bigbly hna warning eno bifks l Optimize for size
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]  
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)